In [2]:
# import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib3
urllib3.disable_warnings()

In [14]:
links = []

print('Getting category urls')
print()
agent = {
    "User-Agent":'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36',
    'referer':'https://www.google.com/'
}

url = "http://books.toscrape.com/catalogue/category/books_1/index.html"
search = requests.get(url, headers=agent)
soup = BeautifulSoup(search.content,'html.parser')

soup = soup.find(class_='nav nav-list')
soup = soup.li.ul

for link in soup.find_all('li'):
    link = link.find('a')
    links.append("http://books.toscrape.com/catalogue/category" + link.get('href')[2:])

    
print()
print('complete')

Getting category urls


complete


In [15]:
links

['http://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'http://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'http://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'http://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'http://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'http://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'http://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'http://books.toscrape.com/catalogue/category/books/religion_12/index.html',
 'http://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html',
 'http://books.toscrape.com/catalogue/category/boo

In [75]:
import re
theme = []
number = []
total_rating = []

for cat in links:
    # get the content 
        page = requests.get(cat)
        soup = BeautifulSoup(page.text)
        # find the theme
        try:
            the = soup.find(class_='page-header action')
            theme.append(the.text.replace('\n', '').replace('\r', ''))
        except:
            theme.append("")
            pass
        # get the number
        try:
            num = soup.find(class_='form-horizontal')
            num = re.findall("\d+",num.text)[0]
            number.append(int(num))
        except:
            number.append(0)
            pass
        
        # get the average_rating
        # determine how many pages we have in every category
        pages = 0
        total_rating += [0]
        try:
            pa = soup.find(class_='current')
            pa = re.findall("\d+",pa.text)[1]
            pages += int(pa)
        except:
            pages += 1
            pass
        
        dic = {
            "Zero":0,
            "One":1,
            "Two":2,
            "Three":3,
            "Four":4,
            "Five":5
        }
        
        for i in range(0,pages):
            the = theme[-1].lower().replace(" ", "-") + "_" + str(len(theme)+1)
            page = "page-"+str(i + 1)
            al = the+"/"+page
            global lis 
            if pages == 1:
                lis = ["http://books.toscrape.com/catalogue/category/books/",the,"/index.html"]
            else:
                lis = ["http://books.toscrape.com/catalogue/category/books/",al,".html"]
            url = ''.join(lis).replace('\n', '').replace('\r', '')
            search = requests.get(url, headers=agent)

            soup = BeautifulSoup(search.content,'html.parser')
            for rat in soup.find_all('li',attrs={'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'}):
                rat = rat.find('p')['class'][1]
                total_rating[-1]+=dic[rat]               

In [76]:
from pandas import DataFrame

my_list = list(zip(theme,number,total_rating))
df = DataFrame(my_list,columns=['Theme','Number_of_Books','Total_rating'])
df['Average_Rating'] = df['Total_rating']/df['Number_of_Books']
df = df.drop(columns=['Total_rating'])
df

,Theme,Number_of_Books,Average_Rating
0,Travel,11,2.727273
1,Mystery,32,2.937500
2,Historical Fiction,26,3.230769
3,Sequential Art,75,2.973333
4,Classics,19,2.473684
5,Philosophy,11,2.363636
6,Romance,35,2.628571
7,Womens Fiction,17,3.117647
8,Fiction,65,3.184615
9,Childrens,29,2.620690


In [ ]:
df.to_csv("/Users/zhouxueqi/Desktop/first.csv")